In [1]:
# torch imports
import torch
import torch._dynamo
torch._dynamo.config.suppress_errors = True
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [2]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    pred_threshold: 0.5 

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

<IPython.core.display.Javascript object>

In [3]:
%%yaml calibration_cfg 

calibrator:
    _name: None 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: 1 
    num_prob_bins: 15
    neighborhood_width: 3

<IPython.core.display.Javascript object>

In [4]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    pred_label: 0 
    _type: "standard"
    pretrained_exp_root : None
    checkpoint: 'min-val-area_estimation_error'
    # checkpoint: 'min-val-ece_loss'

<IPython.core.display.Javascript object>

## Gather Inference Options.

In [5]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "ISLES_3D_Eval"

base_model: 
    - "/storage/vbutoi/scratch/ESE/training/09_15_24_ISLES_3D_Dice_wAUG"
    # - "/storage/vbutoi/scratch/ESE/calibration/09_15_24_ISLES_3D_Dice_LTS_wAUG"

inference_data:
    _class: "ese.datasets.ISLES"
    version: 1.0
    split: ("cal", "val")

dataloader:
    batch_size: 1

<IPython.core.display.Javascript object>

In [6]:
# Local imports
from ese.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [7]:
len(inf_cfgs)

3

## Running Jobs

In [8]:
from ese.analysis.run_inference import get_cal_stats

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mnt/hdd/home/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/storage/vbutoi/libraries/pylot/pandas/register.py:47: UserWarning: registration of accessor <class 'pylot.pandas.register.register_series_method.<locals>.inner.<locals>.Ac

In [9]:
from ese.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[-1], 
    job_func=get_cal_stats,
    run_name='debug',
    # show_examples=True,
    show_examples=True,
    gpu='0',
    # gpu='4',
) 

/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Set seed: 40
Loading checkpoint: min-val-area_estimation_error.
[Errno 2] No such file or directory: '/storage/vbutoi/scratch/ESE/training/09_15_24_ISLES_3D_Dice_wAUG/20240915_140848-JDNK-d2e118380f68db71bd23b4cedfadb19f/checkpoints/min-val-area_estimation_error.pt'
Defaulting to loading: max-val-dice_score.
Running:

calibrator:
  _name: None
dataloader:
  batch_size: 1
  num_workers: 1
  pin_memory: true
experiment:
  exp_name: ISLES_3D_Eval
  exp_root: /storage/vbutoi/scratch/ESE/inference/09_17_24_ISLES_3D_Eval
  inf_dataset_name: ISLES
  inference_seed: 40
  model_dir: /storage/vbutoi/scratch/ESE/training/09_15_24_ISLES_3D_Dice_wAUG/20240915_140848-JDNK-d2e118380f68db71bd23b4cedfadb19f
  pred_threshold: 0.5
  pretrained_seed: 40
global_cal_metrics:
- ECE:
    _fn: ese.metrics.calibration.ece_loss
    cal_type: toplabel
global_calibration:
  neighborhood_width: 3
  num_classes: 1
  num_prob_bins: 15
image_cal_metrics:
- Image_ECE:
    _fn: ese.metrics.calibration.image_ece_loss
   

W0917 15:21:07.537621 140518313678656 torch/_dynamo/variables/builtin.py:775] [0/0] incorrect arg count <bound method BuiltinVariable.call_enumerate of BuiltinVariable()> got an unexpected keyword argument 'start' and no constant handler


W0917 15:21:10.135174 140518313678656 torch/_dynamo/convert_frame.py:1009] WON'T CONVERT forward /storage/vbutoi/projects/ESE/ese/models/unet.py line 84 
W0917 15:21:10.135174 140518313678656 torch/_dynamo/convert_frame.py:1009] due to: 
W0917 15:21:10.135174 140518313678656 torch/_dynamo/convert_frame.py:1009] Traceback (most recent call last):
W0917 15:21:10.135174 140518313678656 torch/_dynamo/convert_frame.py:1009]   File "/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/_dynamo/convert_frame.py", line 948, in __call__
W0917 15:21:10.135174 140518313678656 torch/_dynamo/convert_frame.py:1009]     result = self._inner_convert(
W0917 15:21:10.135174 140518313678656 torch/_dynamo/convert_frame.py:1009]   File "/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/torch/_dynamo/convert_frame.py", line 472, in __call__
W0917 15:21:10.135174 140518313678656 torch/_dynamo/convert_frame.py:1009]     return _compile(
W0917 15:21:10.135174 140518313678656 torch/_dynamo/c

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 180 but got size 148 for tensor number 1 in the list.

In [ ]:
# from ese.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     group="inference",
#     base_cfg=updated_base_cfg,
#     exp_cfg=experiment_cfg,
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
#     # available_gpus=['4', '5', '6', '7']
# )